# Electricity

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'/home/lixna37/Downloads/FDCSs_Feb_2024/Datasets/FDCS_2/FDCS_2_Elec_Feb_2024.csv')

output_names = ['Electricity Consumption']

from FDCS_2_Elec_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train_normalized:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val_normalized:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test_normalized:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val_normalized:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1800, 35)
Shape of y_train_normalized: (1800, 1)
Shape of X_val_normalized: (336, 35)
Shape of y_val_normalized: (336, 1)
Shape of X_test_normalized: (336, 35)
Shape of y_test_normalized: (336, 1)
Shape of X_train_val_normalized: (2136, 35)
Shape of y_train_val_normalized: (2136, 1)


## Models

In [2]:
from FDCS_2_models_FS import run_model
pred_true_TestSet = run_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

Execution time: 314.12 seconds


,Model,True_Electricity Consumption,Pred_Electricity Consumption
0,KNeighborsRegressor (Without),0.487435,0.303733
1,KNeighborsRegressor (Without),0.536867,0.303893
2,KNeighborsRegressor (Without),0.434658,0.306775
3,KNeighborsRegressor (Without),0.542578,0.276825
4,KNeighborsRegressor (Without),0.320490,0.278761
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.570382,0.243701
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.866195,0.289118
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.475943,0.477135
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.728326,0.497559


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True_{output_name}']
        pred_values = temp_df[f'Pred_{output_name}']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
25,RandomForestRegressor (Wrapper - RFE (LinearRe...,Electricity Consumption,0.1145,0.1512
22,MLPRegressor (Embedded - ElasticNet),Electricity Consumption,0.1147,0.1528
5,RandomForestRegressor (Filter - Correlation),Electricity Consumption,0.1151,0.1547
24,KNeighborsRegressor (Wrapper - RFE (LinearRegr...,Electricity Consumption,0.1153,0.1528
1,RandomForestRegressor (Without),Electricity Consumption,0.1154,0.1539
27,XGBRegressor (Wrapper - RFE (LinearRegression)),Electricity Consumption,0.1157,0.1524
9,RandomForestRegressor (Filter - Mutual Informa...,Electricity Consumption,0.1161,0.1550
21,RandomForestRegressor (Embedded - ElasticNet),Electricity Consumption,0.1162,0.1541
17,RandomForestRegressor (Embedded - Tree Importa...,Electricity Consumption,0.1162,0.1550
26,MLPRegressor (Wrapper - RFE (LinearRegression)),Electricity Consumption,0.1164,0.1589


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_elec.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Electricity Consumption'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True_Electricity Consumption']])
pred_true_TestSet['Pred_Electricity Consumption'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred_Electricity Consumption']])
pred_true_TestSet.to_csv('FDCS_2_Elec_pred_true_TestSet.csv')
pred_true_TestSet


,Model,True_Electricity Consumption,Pred_Electricity Consumption
0,KNeighborsRegressor (Without),1638.49,1020.982637
1,KNeighborsRegressor (Without),1804.65,1021.520209
2,KNeighborsRegressor (Without),1461.08,1031.209016
3,KNeighborsRegressor (Without),1823.85,930.531806
4,KNeighborsRegressor (Without),1077.31,937.042534
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,1917.31,819.189815
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,2911.67,971.855273
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,1599.86,1603.864553
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,2448.23,1672.519437


## LSTM

In [2]:

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Set to an invalid value to use CPU

from FDCS_2_LSTM_FS import run_lstm_model

pred_true_TestSet = run_lstm_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

2024-03-07 16:25:07.093873: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 16:25:07.095930: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 16:25:07.136824: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 16:25:07.137738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 16:25:07.851982: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch 1/100


2024-03-07 16:25:09.677459: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 16:25:09.679557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 16:25:09.681014: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/11 [==========================>...] - ETA: 0s - loss: 0.0724

2024-03-07 16:25:12.569671: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 16:25:12.571585: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 16:25:12.572876: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

2024-03-07 16:59:39.157437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 16:59:39.158848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 16:59:39.160360: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

11/11 [==============================] - 3s 97ms/step - loss: 0.0727 - val_loss: 0.0336
Epoch 2/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0257 - val_loss: 0.0375
Epoch 3/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0239 - val_loss: 0.0272
Epoch 4/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0229 - val_loss: 0.0287
Epoch 5/100
11/11 [==============================] - 0s 34ms/step - loss: 0.0201 - val_loss: 0.0300
Epoch 6/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0196 - val_loss: 0.0294
Epoch 7/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0191 - val_loss: 0.0289
Epoch 8/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0190 - val_loss: 0.0279
Epoch 9/100
11/11 [==============================] - 0s 34ms/step - loss: 0.0189 - val_loss: 0.0266
Epoch 10/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0186 - val_loss: 0.0291
Epoch 11/10

2024-03-07 17:03:30.729779: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 17:03:30.731724: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 17:03:30.732999: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 1s 9ms/step
Execution time: 2302.59 seconds


,Model,True,Pred
0,LSTM + Without,0.230314,0.331705
1,LSTM + Without,0.426994,0.247920
2,LSTM + Without,0.297202,0.218416
3,LSTM + Without,0.198780,0.262485
4,LSTM + Without,0.296533,0.239681
...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.570382,0.696688
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.866195,0.529437
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.475943,0.273729
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.728326,0.653441


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True']
        pred_values = temp_df[f'Pred']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
0,LSTM + Without,Electricity Consumption,0.1302,0.1683
1,LSTM + Filter - Correlation,Electricity Consumption,0.1325,0.1805
4,LSTM + Embedded - Tree Importance (Extra Trees),Electricity Consumption,0.1388,0.1878
3,LSTM + Embedded - Lasso,Electricity Consumption,0.1464,0.1852
6,LSTM + Wrapper - RFE (LinearRegression),Electricity Consumption,0.1481,0.1868
2,LSTM + Filter - Mutual Information,Electricity Consumption,0.1482,0.1981
7,LSTM + Wrapper - Forward Selection,Electricity Consumption,0.1485,0.1958
8,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,Electricity Consumption,0.1487,0.1807
5,LSTM + Embedded - ElasticNet,Electricity Consumption,0.1570,0.1999


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_elec.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Electricity Consumption'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True']])
pred_true_TestSet['Pred_Electricity Consumption'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred']])
pred_true_TestSet.to_csv('FDCS_2_Elec_pred_true_TestSet_LSTM.csv')
pred_true_TestSet


,Model,True,Pred,True_Electricity Consumption,Pred_Electricity Consumption
0,LSTM + Without,0.230314,0.331705,774.19,1115.011230
1,LSTM + Without,0.426994,0.247920,1435.32,833.371643
2,LSTM + Without,0.297202,0.218416,999.03,734.193542
3,LSTM + Without,0.198780,0.262485,668.19,882.331726
4,LSTM + Without,0.296533,0.239681,996.78,805.675781
...,...,...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.570382,0.696688,1917.31,2341.881348
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.866195,0.529437,2911.67,1779.675049
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.475943,0.273729,1599.86,920.125610
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.728326,0.653441,2448.23,2196.509521


# Temp

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'/home/lixna37/Downloads/FDCSs_Feb_2024/Datasets/FDCS_2/FDCS_2_Temp_Feb_2024.csv')

output_names = ['Temperature Closer to Evaporator (C)']

from FDCS_2_Temp_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train_normalized:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val_normalized:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test_normalized:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val_normalized:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1800, 35)
Shape of y_train_normalized: (1800, 1)
Shape of X_val_normalized: (336, 35)
Shape of y_val_normalized: (336, 1)
Shape of X_test_normalized: (336, 35)
Shape of y_test_normalized: (336, 1)
Shape of X_train_val_normalized: (2136, 35)
Shape of y_train_val_normalized: (2136, 1)


## Models

In [2]:
from FDCS_2_models_FS import run_model
pred_true_TestSet = run_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

Applying FS method: Without
Number of features before FS: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Number of features selected: 35
Applying FS method: Filter - Correlation
Number of features before FS: 35
Number of features selected: 20
Number of features selected: 20
Number of features selected: 20
Number of features selected: 20
Number of features selected: 20
Number of features selected: 20
Number of features selected: 20
N

,Model,True_Temperature Closer to Evaporator (C),Pred_Temperature Closer to Evaporator (C)
0,KNeighborsRegressor (Without),0.591281,0.520676
1,KNeighborsRegressor (Without),0.408719,0.511187
2,KNeighborsRegressor (Without),0.386921,0.509132
3,KNeighborsRegressor (Without),0.250681,0.483616
4,KNeighborsRegressor (Without),0.362398,0.454848
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.634877,0.488474
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.204360,0.464018
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.136240,0.404634
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.226158,0.440999


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True_{output_name}']
        pred_values = temp_df[f'Pred_{output_name}']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
13,RandomForestRegressor (Embedded - Lasso),Temperature Closer to Evaporator (C),0.1190,0.1580
1,RandomForestRegressor (Without),Temperature Closer to Evaporator (C),0.1192,0.1594
3,XGBRegressor (Without),Temperature Closer to Evaporator (C),0.1201,0.1626
9,RandomForestRegressor (Filter - Mutual Informa...,Temperature Closer to Evaporator (C),0.1201,0.1611
15,XGBRegressor (Embedded - Lasso),Temperature Closer to Evaporator (C),0.1207,0.1632
11,XGBRegressor (Filter - Mutual Information),Temperature Closer to Evaporator (C),0.1219,0.1659
30,MLPRegressor (Wrapper - Forward Selection),Temperature Closer to Evaporator (C),0.1219,0.1684
25,RandomForestRegressor (Wrapper - RFE (LinearRe...,Temperature Closer to Evaporator (C),0.1255,0.1636
2,MLPRegressor (Without),Temperature Closer to Evaporator (C),0.1261,0.1638
5,RandomForestRegressor (Filter - Correlation),Temperature Closer to Evaporator (C),0.1264,0.1628


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_temp.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Temperature Closer to Evaporator (C)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True_Temperature Closer to Evaporator (C)']])
pred_true_TestSet['Pred_Temperature Closer to Evaporator (C)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred_Temperature Closer to Evaporator (C)']])
pred_true_TestSet.to_csv('FDCS_2_Temp_pred_true_TestSet.csv')
pred_true_TestSet


,Model,True_Temperature Closer to Evaporator (C),Pred_Temperature Closer to Evaporator (C)
0,KNeighborsRegressor (Without),10.17,9.910880
1,KNeighborsRegressor (Without),9.50,9.876055
2,KNeighborsRegressor (Without),9.42,9.868516
3,KNeighborsRegressor (Without),8.92,9.774869
4,KNeighborsRegressor (Without),9.33,9.669293
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,10.33,9.792700
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,8.75,9.702945
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,8.50,9.485007
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,8.83,9.618468


## LSTM

In [2]:

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Set to an invalid value to use CPU

from FDCS_2_LSTM_FS import run_lstm_model

pred_true_TestSet = run_lstm_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

2024-03-07 23:05:31.742788: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 23:05:31.744844: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 23:05:31.785357: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 23:05:31.786308: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 23:05:32.489420: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch 1/5


2024-03-07 23:05:34.210377: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 23:05:34.212237: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 23:05:34.213880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/11 [==========================>...] - ETA: 0s - loss: 0.0461

2024-03-07 23:05:37.009313: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 23:05:37.011106: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 23:05:37.012499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

11/11 [==============================] - 4s 108ms/step - loss: 0.0437 - val_loss: 0.0391
Epoch 2/5
11/11 [==============================] - 0s 34ms/step - loss: 0.0166 - val_loss: 0.0378
Epoch 3/5
11/11 [==============================] - 0s 33ms/step - loss: 0.0142 - val_loss: 0.0227
Epoch 4/5
11/11 [==============================] - 0s 34ms/step - loss: 0.0128 - val_loss: 0.0276
Epoch 5/5
11/11 [==============================] - 0s 34ms/step - loss: 0.0125 - val_loss: 0.0394
Epoch 1/5
22/22 [==============================] - 1s 28ms/step - loss: 0.0198 - val_loss: 0.0174
Epoch 2/5
22/22 [==============================] - 1s 27ms/step - loss: 0.0167 - val_loss: 0.0173
Epoch 3/5
22/22 [==============================] - 1s 27ms/step - loss: 0.0169 - val_loss: 0.0188
Epoch 4/5
22/22 [==============================] - 1s 27ms/step - loss: 0.0176 - val_loss: 0.0190
Epoch 5/5
22/22 [==============================] - 1s 27ms/step - loss: 0.0164 - val_loss: 0.0172
Epoch 1/5
33/33 [============

2024-03-07 23:05:58.079860: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 23:05:58.081671: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 23:05:58.082858: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 1s 9ms/step
Execution time: 25.41 seconds


,Model,True,Pred
0,LSTM + Without,0.498638,0.545844
1,LSTM + Without,0.523161,0.511992
2,LSTM + Without,0.455041,0.519936
3,LSTM + Without,0.544959,0.528073
4,LSTM + Without,0.566757,0.535865
...,...,...,...
307,LSTM + Without,0.634877,0.521369
308,LSTM + Without,0.204360,0.537110
309,LSTM + Without,0.136240,0.548416
310,LSTM + Without,0.226158,0.554705


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True']
        pred_values = temp_df[f'Pred']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
3,LSTM + Embedded - Lasso,Temperature Closer to Evaporator (C),0.1205,0.1685
2,LSTM + Filter - Mutual Information,Temperature Closer to Evaporator (C),0.1387,0.1972
6,LSTM + Wrapper - RFE (LinearRegression),Temperature Closer to Evaporator (C),0.1489,0.1986
0,LSTM + Without,Temperature Closer to Evaporator (C),0.1504,0.2124
5,LSTM + Embedded - ElasticNet,Temperature Closer to Evaporator (C),0.1505,0.1869
8,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,Temperature Closer to Evaporator (C),0.1547,0.2103
7,LSTM + Wrapper - Forward Selection,Temperature Closer to Evaporator (C),0.1661,0.2118
1,LSTM + Filter - Correlation,Temperature Closer to Evaporator (C),0.1672,0.2235
4,LSTM + Embedded - Tree Importance (Extra Trees),Temperature Closer to Evaporator (C),0.1707,0.2355


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_temp.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Temperature Closer to Evaporator (C)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True']])
pred_true_TestSet['Pred_Temperature Closer to Evaporator (C)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred']])
pred_true_TestSet.to_csv('FDCS_2_Temp_pred_true_TestSet_LSTM.csv')
pred_true_TestSet


,Model,True,Pred,True_Temperature Closer to Evaporator (C),Pred_Temperature Closer to Evaporator (C)
0,LSTM + Without,0.498638,0.663528,9.83,10.435148
1,LSTM + Without,0.523161,0.535652,9.92,9.965841
2,LSTM + Without,0.455041,0.440353,9.67,9.616096
3,LSTM + Without,0.544959,0.545256,10.00,10.001088
4,LSTM + Without,0.566757,0.667565,10.08,10.449965
...,...,...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.634877,0.757178,10.33,10.778843
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.204360,0.760879,8.75,10.792427
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.136240,0.726292,8.50,10.665491
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.226158,0.721023,8.83,10.646155


# Humd

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


df = pd.read_csv(r'/home/lixna37/Downloads/FDCSs_Feb_2024/Datasets/FDCS_2/FDCS_2_Humd_Feb_2024.csv')

output_names = ['Humidity Closer to Evaporator (%)']

from FDCS_2_Humd_pre_processing import preprocess_data
X_train_normalized, y_train_normalized, X_val_normalized, y_val_normalized, X_test_normalized, y_test_normalized = preprocess_data(df)

# Combine the training and validation sets directly after they have been recalled
X_train_val_normalized = np.concatenate([X_train_normalized, X_val_normalized], axis=0)
y_train_val_normalized = np.concatenate([y_train_normalized, y_val_normalized], axis=0)

print("Shape of X_train_normalized:", X_train_normalized.shape)
print("Shape of y_train_normalized:", y_train_normalized.shape)
print("Shape of X_val_normalized:", X_val_normalized.shape)
print("Shape of y_val_normalized:", y_val_normalized.shape)
print("Shape of X_test_normalized:", X_test_normalized.shape)
print("Shape of y_test_normalized:", y_test_normalized.shape)
print("Shape of X_train_val_normalized:", X_train_val_normalized.shape)
print("Shape of y_train_val_normalized:", y_train_val_normalized.shape)

Shape of X_train_normalized: (1800, 35)
Shape of y_train_normalized: (1800, 1)
Shape of X_val_normalized: (336, 35)
Shape of y_val_normalized: (336, 1)
Shape of X_test_normalized: (336, 35)
Shape of y_test_normalized: (336, 1)
Shape of X_train_val_normalized: (2136, 35)
Shape of y_train_val_normalized: (2136, 1)


## Models

In [2]:
from FDCS_2_models_FS import run_model
pred_true_TestSet = run_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

Execution time: 303.92 seconds


,Model,True_Humidity Closer to Evaporator (%),Pred_Humidity Closer to Evaporator (%)
0,KNeighborsRegressor (Without),0.651840,0.670254
1,KNeighborsRegressor (Without),0.434433,0.670168
2,KNeighborsRegressor (Without),0.495015,0.670905
3,KNeighborsRegressor (Without),0.456672,0.672674
4,KNeighborsRegressor (Without),0.460123,0.678541
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.575153,0.599212
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.242715,0.595840
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.245782,0.589278
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,0.303298,0.529645


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True_{output_name}']
        pred_values = temp_df[f'Pred_{output_name}']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)


,Model_FS,Output,MAE,RMSE
22,MLPRegressor (Embedded - ElasticNet),Humidity Closer to Evaporator (%),0.1107,0.1469
26,MLPRegressor (Wrapper - RFE (LinearRegression)),Humidity Closer to Evaporator (%),0.1112,0.1462
25,RandomForestRegressor (Wrapper - RFE (LinearRe...,Humidity Closer to Evaporator (%),0.1132,0.1499
34,MLPRegressor (Hybrid - Filter (SelectKBest) + ...,Humidity Closer to Evaporator (%),0.1132,0.1529
33,RandomForestRegressor (Hybrid - Filter (Select...,Humidity Closer to Evaporator (%),0.1138,0.1502
17,RandomForestRegressor (Embedded - Tree Importa...,Humidity Closer to Evaporator (%),0.1138,0.1491
21,RandomForestRegressor (Embedded - ElasticNet),Humidity Closer to Evaporator (%),0.1156,0.1497
19,XGBRegressor (Embedded - Tree Importance (Extr...,Humidity Closer to Evaporator (%),0.1159,0.1521
9,RandomForestRegressor (Filter - Mutual Informa...,Humidity Closer to Evaporator (%),0.1161,0.1523
35,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,Humidity Closer to Evaporator (%),0.1162,0.1529


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_humd.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Humidity Closer to Evaporator (%)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True_Humidity Closer to Evaporator (%)']])
pred_true_TestSet['Pred_Humidity Closer to Evaporator (%)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred_Humidity Closer to Evaporator (%)']])
pred_true_TestSet.to_csv('FDCS_2_Humd_pred_true_TestSet.csv')
pred_true_TestSet


,Model,True_Humidity Closer to Evaporator (%),Pred_Humidity Closer to Evaporator (%)
0,KNeighborsRegressor (Without),76.67,77.150218
1,KNeighborsRegressor (Without),71.00,77.147986
2,KNeighborsRegressor (Without),72.58,77.167209
3,KNeighborsRegressor (Without),71.58,77.213345
4,KNeighborsRegressor (Without),71.67,77.366353
...,...,...,...
331,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,74.67,75.297441
332,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,66.00,75.209502
333,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,66.08,75.038370
334,XGBRegressor (Hybrid - Filter (SelectKBest) + ...,67.58,73.483147


## LSTM

In [2]:

#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Set to an invalid value to use CPU

from FDCS_2_LSTM_FS import run_lstm_model

pred_true_TestSet = run_lstm_model(X_train_val_normalized, y_train_val_normalized, X_test_normalized, y_test_normalized, output_names)
pred_true_TestSet

2024-03-07 17:05:37.406726: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-07 17:05:37.408915: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 17:05:37.449397: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-07 17:05:37.450229: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 17:05:38.159053: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Epoch 1/100


2024-03-07 17:05:40.008036: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 17:05:40.009877: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 17:05:40.011155: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/11 [==========================>...] - ETA: 0s - loss: 0.0600

2024-03-07 17:05:43.015172: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 17:05:43.017085: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 17:05:43.018432: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

2024-03-07 17:42:01.194833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 17:42:01.196662: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 17:42:01.197911: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

11/11 [==============================] - 3s 104ms/step - loss: 0.0883 - val_loss: 0.0340
Epoch 2/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0240 - val_loss: 0.0234
Epoch 3/100
11/11 [==============================] - 0s 32ms/step - loss: 0.0203 - val_loss: 0.0232
Epoch 4/100
11/11 [==============================] - 0s 32ms/step - loss: 0.0199 - val_loss: 0.0235
Epoch 5/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0200 - val_loss: 0.0231
Epoch 6/100
11/11 [==============================] - 0s 32ms/step - loss: 0.0193 - val_loss: 0.0233
Epoch 7/100
11/11 [==============================] - 0s 32ms/step - loss: 0.0196 - val_loss: 0.0240
Epoch 8/100
11/11 [==============================] - 0s 34ms/step - loss: 0.0195 - val_loss: 0.0229
Epoch 9/100
11/11 [==============================] - 0s 34ms/step - loss: 0.0191 - val_loss: 0.0229
Epoch 10/100
11/11 [==============================] - 0s 33ms/step - loss: 0.0192 - val_loss: 0.0227
Epoch 11/1

2024-03-07 17:46:39.296377: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-07 17:46:39.298369: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-07 17:46:39.299943: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10/10 [==============================] - 1s 8ms/step
Execution time: 2460.83 seconds


,Model,True,Pred
0,LSTM + Without,0.616564,0.720211
1,LSTM + Without,0.565567,0.723749
2,LSTM + Without,0.629218,0.716154
3,LSTM + Without,0.718942,0.585501
4,LSTM + Without,0.728528,0.455724
...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.575153,0.569375
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.242715,0.562009
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.245782,0.542252
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.303298,0.583802


In [3]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate MAE and RMSE for each FS method and model
results = []
for model_fs in pred_true_TestSet['Model'].unique():
    temp_df = pred_true_TestSet[pred_true_TestSet['Model'] == model_fs]
    for output_name in output_names:
        true_values = temp_df[f'True']
        pred_values = temp_df[f'Pred']
        mae = mean_absolute_error(true_values, pred_values)
        rmse = np.sqrt(mean_squared_error(true_values, pred_values))
        results.append({'Model_FS': model_fs, 'Output': output_name, 'MAE': mae, 'RMSE': rmse})

meterc_df = pd.DataFrame(results)

# Sort the DataFrame by the "MAE" column in ascending order
meterc_df = meterc_df.sort_values(by='MAE')
meterc_df.round(4)

,Model_FS,Output,MAE,RMSE
3,LSTM + Embedded - Lasso,Humidity Closer to Evaporator (%),0.1304,0.1684
5,LSTM + Embedded - ElasticNet,Humidity Closer to Evaporator (%),0.1319,0.1746
2,LSTM + Filter - Mutual Information,Humidity Closer to Evaporator (%),0.1324,0.1757
8,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,Humidity Closer to Evaporator (%),0.1330,0.1699
4,LSTM + Embedded - Tree Importance (Extra Trees),Humidity Closer to Evaporator (%),0.1375,0.1791
1,LSTM + Filter - Correlation,Humidity Closer to Evaporator (%),0.1415,0.1814
7,LSTM + Wrapper - Forward Selection,Humidity Closer to Evaporator (%),0.1422,0.1843
6,LSTM + Wrapper - RFE (LinearRegression),Humidity Closer to Evaporator (%),0.1523,0.1959
0,LSTM + Without,Humidity Closer to Evaporator (%),0.1714,0.2160


In [4]:
import joblib
import pandas as pd
# Load the saved scaler
scaler_y_loaded = joblib.load('scaler_y_humd.joblib')
# Apply inverse transformation to the true and predicted values
pred_true_TestSet['True_Humidity Closer to Evaporator (%)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['True']])
pred_true_TestSet['Pred_Humidity Closer to Evaporator (%)'] = scaler_y_loaded.inverse_transform(pred_true_TestSet[['Pred']])
pred_true_TestSet.to_csv('FDCS_2_Humd_pred_true_TestSet_LSTM.csv')
pred_true_TestSet

,Model,True,Pred,True_Humidity Closer to Evaporator (%),Pred_Humidity Closer to Evaporator (%)
0,LSTM + Without,0.616564,0.720211,75.75,78.453110
1,LSTM + Without,0.565567,0.723749,74.42,78.545372
2,LSTM + Without,0.629218,0.716154,76.08,78.347290
3,LSTM + Without,0.718942,0.585501,78.42,74.939865
4,LSTM + Without,0.728528,0.455724,78.67,71.555275
...,...,...,...,...,...
2803,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.575153,0.569375,74.67,74.519295
2804,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.242715,0.562009,66.00,74.327209
2805,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.245782,0.542252,66.08,73.811928
2806,LSTM + Hybrid - Filter (SelectKBest) + Embedde...,0.303298,0.583802,67.58,74.895569
